In [1]:
import os
import random
import shutil

import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [4]:
!pip install kaggle

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shaunthesheep/microsoft-catsvsdogs-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'microsoft-catsvsdogs-dataset' dataset.
Path to dataset files: /kaggle/input/microsoft-catsvsdogs-dataset


In [6]:
import os

base_path = path  # from kagglehub
print(os.listdir(base_path))

['PetImages', 'readme[1].txt', 'MSR-LA - 3467.docx']


In [7]:
pet_path = os.path.join(base_path, "PetImages")
print(os.listdir(pet_path))

['Dog', 'Cat']


In [8]:
pet_path = os.path.join(path, "PetImages")
target_raw = "/content/dataset_raw"

if not os.path.exists(target_raw):
    shutil.copytree(pet_path, target_raw)

print("Dataset copied to writable directory")


Dataset copied to writable directory


In [9]:
def clean_images(folder):
    removed = 0
    for cls in ["Cat", "Dog"]:
        cls_path = os.path.join(folder, cls)
        for img in os.listdir(cls_path):
            img_path = os.path.join(cls_path, img)

            if not img.lower().endswith((".jpg", ".jpeg", ".png")):
                continue

            try:
                Image.open(img_path).verify()
            except:
                os.remove(img_path)
                removed += 1

    print("Removed corrupted images:", removed)

clean_images("/content/dataset_raw")


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Removed corrupted images: 2


In [10]:
SOURCE = "/content/dataset_raw"
TARGET = "/content/dataset"
VALID_EXT = (".jpg", ".jpeg", ".png")

# clean old split if exists
shutil.rmtree(TARGET, ignore_errors=True)

for split in ["train", "val"]:
    for cls in ["cat", "dog"]:
        os.makedirs(f"{TARGET}/{split}/{cls}", exist_ok=True)

def split_and_copy(cls):
    src = os.path.join(SOURCE, cls.capitalize())
    images = [f for f in os.listdir(src) if f.lower().endswith(VALID_EXT)]
    random.shuffle(images)

    cut = int(0.8 * len(images))
    train_imgs, val_imgs = images[:cut], images[cut:]

    for img in train_imgs:
        shutil.copy(src + "/" + img, f"{TARGET}/train/{cls}/{img}")
    for img in val_imgs:
        shutil.copy(src + "/" + img, f"{TARGET}/val/{cls}/{img}")

split_and_copy("cat")
split_and_copy("dog")

print("Train/Val split completed")


Train/Val split completed


In [11]:
!ls dataset/train
!ls dataset/val


cat  dog
cat  dog


In [13]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder("dataset/train", transform=train_transform)
val_data   = datasets.ImageFolder("dataset/val", transform=val_transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False)

print("Classes:", train_data.classes)


Classes: ['cat', 'dog']


In [17]:
train_data = datasets.ImageFolder("dataset/train", transform=train_transform)
val_data   = datasets.ImageFolder("dataset/val", transform=val_transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False)

print("Classes:", train_data.classes)


Classes: ['cat', 'dog']


In [14]:
model = models.efficientnet_b0(pretrained=True)

model.classifier[1] = nn.Linear(
    model.classifier[1].in_features,
    2
)

model = model.to(device)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 150MB/s]


In [15]:
for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.0001
)


In [16]:
def train_one_epoch(model, loader):
    model.train()
    total_loss, correct = 0, 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()

    return total_loss / len(loader), correct / len(loader.dataset)


def validate(model, loader):
    model.eval()
    correct = 0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            correct += (out.argmax(1) == y).sum().item()

    return correct / len(loader.dataset)


In [22]:
epochs = 5

for epoch in range(epochs):
    loss, train_acc = train_one_epoch(model, train_loader)
    val_acc = validate(model, val_loader)

    print(
        f"[EfficientNet] Epoch {epoch+1}/{epochs} | "
        f"Loss: {loss:.4f} | "
        f"Train Acc: {train_acc:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )


[EfficientNet] Epoch 1/5 | Loss: 0.0146 | Train Acc: 0.9949 | Val Acc: 0.9922
[EfficientNet] Epoch 2/5 | Loss: 0.0112 | Train Acc: 0.9963 | Val Acc: 0.9902
[EfficientNet] Epoch 3/5 | Loss: 0.0100 | Train Acc: 0.9965 | Val Acc: 0.9902
[EfficientNet] Epoch 4/5 | Loss: 0.0062 | Train Acc: 0.9978 | Val Acc: 0.9930
[EfficientNet] Epoch 5/5 | Loss: 0.0063 | Train Acc: 0.9982 | Val Acc: 0.9910


In [23]:
def final_evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total


In [24]:
final_accuracy = final_evaluate(model, val_loader)
print(f"Final Validation Accuracy: {final_accuracy:.4f}")


Final Validation Accuracy: 0.9910


In [26]:
from sklearn.metrics import classification_report


In [28]:
import numpy as np

def get_predictions(model, loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            preds = outputs.argmax(1).cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    return np.array(all_preds), np.array(all_labels)


In [29]:
preds, labels = get_predictions(model, val_loader)


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [30]:
print(classification_report(labels, preds, target_names=train_data.classes))


              precision    recall  f1-score   support

         cat       1.00      0.99      0.99      2500
         dog       0.99      1.00      0.99      2500

    accuracy                           0.99      5000
   macro avg       0.99      0.99      0.99      5000
weighted avg       0.99      0.99      0.99      5000



In [31]:
torch.save(model.state_dict(), "efficientnet_part3.pth")
print("Model saved successfully.")


Model saved successfully.


In [32]:
from PIL import Image
import torchvision.transforms as transforms
import os
import random

# Pick random validation image
cat_images = os.listdir("dataset/val/cat")
random_image = random.choice(cat_images)

image_path = f"dataset/val/cat/{random_image}"
print("Testing image:", random_image)

image = Image.open(image_path).convert("RGB")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

image = transform(image).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    output = model(image)
    prediction = output.argmax(1).item()

print("Predicted Class:", train_data.classes[prediction])


Testing image: 3182.jpg
Predicted Class: cat
